In [6]:
import markdown
import math
import time
import numpy as np
import scipy.stats as stats
import pandas as pd
from collections import defaultdict
from timeit import Timer
from math import factorial

<h2>Question 0</h2>
<p>the url for 2009 micro data is <a href="https://www.eia.gov/consumption/residential/data/2009/csv/recs2009_public.csv">https://www.eia.gov/consumption/residential/data/2009/csv/recs2009_public.csv</a><br />
the url for 2015 micro data is <a href="https://www.eia.gov/consumption/residential/data/2015/csv/recs2015_public_v4.csv">https://www.eia.gov/consumption/residential/data/2015/csv/recs2015_public_v4.csv</a><br />
the url for 2009 replicate weights is <a href="https://www.eia.gov/consumption/residential/data/2009/csv/recs2009_public_repweights.csv">https://www.eia.gov/consumption/residential/data/2009/csv/recs2009_public_repweights.csv</a><br />
my variables contain DOEID, NWEIGHT, REGIONC, HDD65, CDD65<br />
the link of how to use replicate weight is <a href="https://www.eia.gov/consumption/residential/methodology/2009/pdf/using-microdata-022613.pdf">https://www.eia.gov/consumption/residential/methodology/2009/pdf/using-microdata-022613.pdf</a>  </p>

<p>$$\hat{V}(\hat{\theta}) = \frac{1}{R(1 - \varepsilon)^2} \sum_{r=1}^{R}(\hat{\theta}_r-\hat{\theta})^2$$<br />
$\theta$ :population parameter of interest<br />
$\hat{\theta}$ :estimate from the full sample for $\theta$<br />
$\hat{\theta}_r$ :estimate from the rth replicate subsample by using replicate weights
$\varepsilon$ : Fay coefficient, 0 &lt; $\varepsilon$ &lt; 1<br />
For the 2009 RECS, R = 244( the number of replicate subsamples) and $\varepsilon$ = 5. The formula for calculating the RSE is:
$$(\frac{\sqrt{\hat{V}(\hat{\theta})}}{\hat{\theta}}) * 100$$</p>

In [8]:
##Q1
##A
data_2009 = pd.read_csv('https://www.eia.gov/consumption/residential/data/2009/csv/recs2009_public.csv')
data_2015 = pd.read_csv('https://www.eia.gov/consumption/residential/data/2015/csv/recs2015_public_v4.csv')
data_r2009 = pd.read_csv('https://www.eia.gov/consumption/residential/data/2009/csv/recs2009_public_repweights.csv')
data_1 = data_2009[['DOEID', 'NWEIGHT', 'REGIONC', 'HDD65', 'CDD65']]
data_2 = data_2015[['DOEID', 'NWEIGHT', 'REGIONC', 'HDD65', 'CDD65']]
data_1[['DOEID','REGIONC']] = data_1[['DOEID','REGIONC']].astype('category')
data_2[['DOEID','REGIONC']] = data_2[['DOEID','REGIONC']].astype('category')


F:\umich\python\ana1\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (717,718) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
F:\umich\python\ana1\lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [11]:
##B
data_3 = data_r2009.drop(['NWEIGHT'], axis = 1)
data_2009_long = pd.melt(data_3, id_vars = ['DOEID'], value_name='val')
data_2009_long.columns = ['ID', '2009_brr_weight', 'replicate_weight']
data_2009_long

,ID,2009_brr_weight,replicate_weight
0,1,brr_weight_1,3628.305158
1,2,brr_weight_1,4408.308000
2,3,brr_weight_1,13664.732100
3,4,brr_weight_1,9456.435434
4,5,brr_weight_1,8659.554041
...,...,...,...
2948247,12079,brr_weight_244,15669.237850
2948248,12080,brr_weight_244,3976.693639
2948249,12081,brr_weight_244,2479.082636
2948250,12082,brr_weight_244,17663.466590


In [10]:
data_4 = data_2015.filter(regex='BRRWT')
data_5 = data_2015[['DOEID']]
data_6 = pd.concat([data_4,data_5],axis=1)
for xx in range(1, 96):
    if xx < 10:
        xx = str(xx)
        xx='0' + xx
    else:
        xx = str(xx)
    a = 'BRRWT{}'.format(xx)

data_2015_long = pd.melt(data_6,id_vars=['DOEID'], value_name = 'values')
data_2015_long.columns = ['ID', '2015_brr_weight', 'replicate_weight']
data_2015_long


,ID,2015_brr_weight,replicate_weight
0,10001,BRRWT1,16555.311270
1,10002,BRRWT1,21486.287200
2,10003,BRRWT1,12297.436760
3,10004,BRRWT1,18553.958900
4,10005,BRRWT1,8075.744168
...,...,...,...
545851,15682,BRRWT96,20183.445250
545852,15683,BRRWT96,19681.916720
545853,15684,BRRWT96,29148.112150
545854,15685,BRRWT96,20695.042680
